# Choosing Concurrency and Parallelism for Your Python Projects
Keywords: concurrency, parallelism, asynchronous, multithreading, multiprocessing, asyncio, process, thread, task

## Table of Contents
* Introduction (1 mins)
* I/O-bound vs CPU-bound (3 mins)
* Concurrency and parallelism demystified (2 mins)
* Pre-emptive multitasking vs cooperative multitasking (3 mins)
* Concurrency in (C)Python (3 mins)
* Race condition (3 mins)
* GIL in CPython (1 mins)
* Multithreading and examples (7 mins)
* Asyncio and examples (7 mins)
* Multiprocessing and examples (7 mins)
* Summary
    * Choosing concurrency and parallelism for your Python projects (2 mins)
    * Concurrency and parallelism packages (1 mins)
* References

## Introduction

> _Dealing with concurrency becomes hard when we lack the ‘working knowledge’ and best practices are not followed._ ‒ Ramith Jayasinghe, Experienced Software Engineer.

A brief description about the generic terms of concurrency and parallelism, multitasking styles, and task bounds to first make understand the basic concepts of concurrency and parallelism will be given in this material. The story of Python interpreter implementation for embracing concurrency will also be given. Popular Python built-in packages for concurrent programming (i.e. multithreading, multiprocessing, and asyncio) will be shown, given some minimum working example. At last by this material, choosing what is the best concurrent programming approach for many projects and extending into more wild 3rd party packages for the sake of concurrency will be easier.

The rest of this material (I mean, all) will be given in Bahasa Indonesia.

> _Written by :  
Muhammad Shalahuddin Yahya Sunarko  
AI Technical Lead of <a href="https://www.qlue.co.id/">Qlue Performa Indonesia</a>  
Jakarta, 21 November 2019 for <a href="https://pycon.id/">PyCon Indonesia 2019</a>_  

### Limitations
Python version >=3.5 is required  
Tested on Python 3.6.8

## I/O-bound vs CPU-bound
Program I/O-bound adalah program yang memiliki kecenderungan lebih banyak menunggu operasi _input_ dan _output._ _Task_ yang berhubungan dengan membaca atau menulis data dari/ke berkas lokal maupun _socket_ dalam jaringan di mana CPU hanya menunggu _task_ tersebut selesai adalah _task_ yang terkait dengan I/O. _Task_ tersebut dilakukan oleh perangkat-perangkat keras di luar CPU seperti misalnya harddisk dan WiFi, untuk memindahkan data dari/ke RAM. Operasi pemindahan tersebut dikenal sebagai IOwait. Pada saat itu, CPU berhenti melakukan eksekusi pada _thread_ tersebut, menunggu I/O selesai, dan CPU dapat dipekerjakan untuk _task_ lain.

Program CPU-bound adalah program yang memiliki kecenderungan lebih banyak melakukan operasi yang mengutilisasi CPU. _Task_ yang berhubungan dengan utilisasi CPU untuk melakukan operasi, misalnya untuk melakukan operasi komputasi hasil perkalian matriks dua buah _array,_ atau operasi perintah memulai pemindahan data dari RAM ke harddisk. Pada jenis _task_ ini, jika CPU berhenti melakukan eksekusi pada _thread_ tersebut, operasi tidak dapat berjalan.

<img src='src/images/1-s2.0-S0045790616302270-gr1.jpg' width='640px'/>
<p style="text-align: center;color:gray">Gambar 1. CPU-bound vs I/O-bound (Sumber: <a href="https://ars.els-cdn.com/content/image/1-s2.0-S0045790616302270-gr1.jpg">https://ars.els-cdn.com/content/image/1-s2.0-S0045790616302270-gr1.jpg</a>)</i></p>


## Concurrency and Parallelism Demystified
_Concurrency_ adalah _task_ yang dapat dimulai, dikerjakan, dan diselesaikan dalam waktu yang bertumpang-tindih dengan berbagai _task_ lain.  
Pengerjaan _task_ tidak musti terjadi dalam waktu yang benar-benar bersamaan (contoh: _multitasking_ pada sistem _single-core_ CPU).

_Parallelism_ adalah _task_ yang dikerjakan dalam waktu yang benar-benar bersamaan. _Parallelism_ dapat dicapai menggunakan sistem _multi-core_ CPU.

Makin banyak core makin besar parallelism. Makin besar kapasitas pemrosesan CPU makin cepat concurrency? Tunggu dulu.

<img src='src/images/parallel_sequential_concurrent.jpg' width='640px'/>
<p style="text-align: center;color:gray">Gambar 2. Ilustrasi pengerjaan <i>task</i> secara <i>sequential</i>, <i>concurrent</i>, dan <i>parallel</i>.<br /><i>(Sumber: <a href="http://www.dietergalea.com/parallelism-concurrency/">http://www.dietergalea.com/parallelism-concurrency/</a> ; dimodifikasi)</i></p>


## Pre-emptive Multitasking vs Cooperative Multitasking

_Pre-emptive multitasking_ adalah _multitasking_ di mana sistem operasi kapanpun dapat menginterupsi pekerjaan suatu _thread_ untuk mengerjakan _thread_ lain. Sistem operasi menggunakan kriteria tertentu untuk menentukan kapan _switch_ pengerjaan _thread,_ salah satunya yang paling umum digunakan adalah _time sharing / time slicing,_ yaitu berapa lama waktu yang telah berlalu. Proses penyimpanan keadaan _thread_ sehingga nantinya dapat kembali dikerjakan pada keadaan yang sama dinamakan _context switch_. _Pre-emptive multitasking_ mudah karena tidak dibutuhkan apapun pada kode dalam _thread_ untuk melakukan _switch_.

_Cooperative multitasking (non-preemptive multitasking)_ adalah _multitasking_ dimana setiap _task_ secara _voluntarily_ / kooperatif memberitahu dengan cara _yield_ (menyerahkan) kontrol pada saat tidak ada pekerjaan atau _blocked,_ kapan bisa dilakukan _switch._ Keuntungan _cooperative multitasking_ yaitu diketahui dengan jelas kapan terjadinya _switch task._ _Switch task_ tidak terjadi pada sembarang kode pada _task_ kecuali kode tersebut ditandai.

<img src='src/images/Cooperative+vs.+Preemptive+Multitasking.jpg' width='640px'/>
<p style="text-align: center;color:gray">Gambar 3. Ilustrasi <i>pre-emptive multitasking</i> dan <i>cooperative multitasking</i>.<br /><i>(Sumber: <a href="https://slideplayer.com/slide/8851057/">https://slideplayer.com/slide/8851057/</a>)</i></p>

## Concurrency in (C)Python
> _Python is actually a specification for a language that can be implemented in many different ways._ ‒ Kenneth Reitz & Real Python

Terdapat berbagai macam implementasi interpreter Python, beberapa di antaranya yang terkenal CPython (C), PyPy (RPython), Jython (Java), dan IronPython (.NET). Setiap interpreter memiliki implementasi masing-masing untuk menginterpretasikan kode Python; implementasi _concurrency_ juga tergantung dengan implementasi interpreter. Sebagai contoh, pada CPython terdapat _Global Interpreter Lock_ / GIL (nantinya akan dibahas), sedangkan pada Jython tidak terdapat GIL. Pada materi ini dipilih interpreter CPython karena interpreter ini merupakan interpreter yang paling umum dan popular digunakan saat ini.

Beberapa _package_ bawaan _(built-in)_ yang digunakan dalam materi ini untuk mencapai _concurrency_ pada Python antara lain `multithreading` _(pre-emptive multitasking),_ `asyncio` _(cooperative multitasking),_ dan `multiprocessing` _(the true parallelism)._ Pada CPython, _concurrency_ pada `multithreading` dan `asyncio` terjadi hanya pada 1 _core_ dari CPU. Sedangkan, _concurrency_ pada `multiprocessing`dapat terjadi pada multi _core_ dari CPU. 

## Race Condition
Race condition terjadi ketika terdapat 2 _thread_ yang melakukan baca dan tulis pada suatu variabel secara bersamaan, sehingga menyebabkan _bug_ pada saat eksekusi.  
Misalkan terdapat 2 buah _thread_ yang berjalan bersamaan tanpa proses sinkronisasi atau _lock_ untuk menambahkan nilai suatu variabel sebesar 1 dari nilai sebelumnya, seperti yang dapat dilihat di bawah ini.

| Thread 1      | Thread 2      |         | Nilai integer |
|---------------|---------------|---------|---------------|
|               |               |         | 0             |
| baca nilai    |               | &#8592; | 0             |
|               | baca nilai    | &#8592; | 0             |
| tambahkan 1   |               |         | 0             |
|               | tambahkan 1   |         | 0             |
| tulis kembali |               | &#8594; | 1             |
|               | tulis kembali | &#8594; | 1             |

<p style="text-align: center;color:gray">Tabel 1. Ilustrasi <i>race condition</i>.<br /><i>(Referensi tabel: <a href="https://en.wikipedia.org/wiki/Race_condition">https://en.wikipedia.org/wiki/Race_condition/</a>)</i></p>

Variabel memiliki nilai awal sebesar 0. Ada 2 _thread_ yang berjalan dan masing-masing _thread_ menambahkan senilai 1 dari nilai sebelumnya. Hasil akhir yang diharapkan variabel memiliki nilai sebesar 2, namun karena adanya _race condition,_ hasil akhir yang didapatkan yaitu variabel memiliki nilai sebesar 1.

Masalah ini dapat diatasi dengan salah satu cara sinkronisasi menggunakan _lock._ _Lock_ memiliki 2 _state:_ _locked_ dan _unlocked._ _Lock_ bisa didapatkan _(acquire)_ dan dilepaskan _(release)_ oleh suatu _thread_ yang sedang berjalan.  
Misalkan pada contoh _race condition_ di atas diberikan sebuah _lock_ yang dibagikan ke kedua _thread._ _Thread_ 1 _acquire lock_ sebelum membaca nilai variabel, sehingga _state lock_ berubah menjadi _locked._ Sesaat kemudian, _thread_ 2 melakukan _acquire lock,_ namun karena _lock_ berada pada _state locked,_ maka _thread_ 2 berhenti dan menunggu hingga _lock_ tersebut berubah _state_ menjadi _unlocked_. _Thread_ 1 melanjutkan eksekusi, yaitu menambahkan nilai 1 dan menuliskan hasilnya ke variabel, kemudian _release_ lock. Karena _lock_ sudah berubah kembali menjadi _unlocked_, maka _thread_ 2 berhasil _acquire lock_, kemudian mekanisme di atas berulang.

| Thread 1      | Thread 2      |         | Nilai integer | Lock state |
|---------------|---------------|---------|---------------|------------|
|               |               |         | 0             | unlocked   |
| acquire lock  |               |         | 0             | unlocked   |
| lock acquired |               |         | 0             | locked     |
|               | acquire lock  |         | 0             | locked     |
| baca nilai    |               | &#8592; | 0             | locked     |
| tambahkan 1   |               |         | 0             | locked     |
| tulis kembali |               | &#8594; | 1             | locked     |
| release lock  |               |         | 1             | locked     |
| lock released |               |         | 1             | unlocked   |
|               | lock acquired |         | 1             | locked     |
|               | baca nilai    | &#8592; | 1             | locked     |
|               | tambahkan 1   |         | 1             | locked     |
|               | tulis kembali | &#8594; | 2             | locked     |
|               | release lock  |         | 2             | locked     |
|               | lock released |         | 2             | unlocked   |

<p style="text-align: center;color:gray">Tabel 2. Ilustrasi mekanisme <i>lock</i>.<br /></p>

## Global Interpreter Lock in CPython

CPython menggunakan _reference counting_ untuk manajemen memori, seluruh objek memiliki _reference count_ dan jika _reference count_ mencapai 0, maka memori yang digunakan oleh objek tersebut akan dibebaskan oleh interpreter.  
Jika _race condition_ seperti pada contoh yang  sebelumnya telah diberikan terjadi pada mekanisme _reference counting_ ini, maka dapat terjadi masalah di antaranya <a href="https://en.wikipedia.org/wiki/Memory_leak"><i>memory leak</i></a> atau memori dibebaskan sedangkan sebenarnya masih digunakan. Oleh karena itu, CPython memberikan solusi sebuah Global Interpreter Lock (GIL) yang digunakan oleh seluruh _thread_ pada suatu _process_ dalam interpreter CPython ketika akan mengeksekusi baris-baris kode perintah.

## Multithreading in CPython
_Package_ `threading` merupakan _built-in package_ dari (C)Python yang dapat digunakan untuk melakukan "*multithreading*". Wajib diketahui, _multithreading_ pada interpreter CPython hanya menggunakan **1 _core_ CPU**, meskipun prosesor yang digunakan memiliki banyak *core*. Hal ini dikarenakan adanya implementasi Global Interpreter Lock pada CPython, seperti pada yang sudah dibahas sebelumnya. Oleh karena itu, *multithreading* pada CPython kurang cocok untuk program yang bersifat CPU-bound dan lebih cocok untuk program yang bersifat I/O-bound.  
Dokumentasi lengkapnya dapat diakses di https://docs.python.org/3.6/library/threading.html.

Terdapat 2 cara untuk memulai *thread* baru:
1. Menimpa method `run()` dari sub-kelas `threading.Thread` (*inheritance*)
2. Memberikan *callable object* ke *keyword argument* `target` dari konstruktor objek / saat membuat objek dari kelas `threading.Thread`

In [1]:
# install requirements
!pip3 install numpy requests aiohttp flask

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Versi sekuensial untuk perbandingan
!python3 src/python/hello_world.py

Finished with 12.753k operations, about 12.752k operation per second


In [3]:
%%timeit -r 3 -n 3 -q
!python3 src/python/hello_world.py

Finished with 12.751k operations, about 12.750k operation per second
Finished with 12.515k operations, about 12.514k operation per second
Finished with 12.26k operations, about 12.259k operation per second
Finished with 12.389k operations, about 12.388k operation per second
Finished with 12.636k operations, about 12.635k operation per second
Finished with 12.588k operations, about 12.587k operation per second
Finished with 12.625k operations, about 12.624k operation per second
Finished with 12.623k operations, about 12.622k operation per second
Finished with 12.618k operations, about 12.617k operation per second


In [4]:
# Berikut dapat dilihat contoh untuk cara threading yang pertama:
!python3 src/python/hello_world_mt.py

Thread-2 finished with 1.0k operations, about 1.000k operations per second
Thread-4 finished with 0.99k operations, about 0.990k operations per second
Thread-3 finished with 0.915k operations, about 0.915k operations per second
Thread-1 finished with 0.9k operations, about 0.900k operations per second
Thread-5 finished with 1.029k operations, about 1.029k operations per second
Thread-6 finished with 1.0k operations, about 1.000k operations per second
Finished all jobs, totalling 5.834k operations, about 5.820k operations per second


In [5]:
%%timeit -r 3 -n 3 -q
!python3 src/python/hello_world_mt.py -q

Finished all jobs, totalling 8.314k operations, about 8.308k operations per second
Finished all jobs, totalling 7.773k operations, about 7.765k operations per second
Finished all jobs, totalling 4.132k operations, about 4.124k operations per second
Finished all jobs, totalling 7.031k operations, about 7.024k operations per second
Finished all jobs, totalling 8.702k operations, about 8.693k operations per second
Finished all jobs, totalling 7.99k operations, about 7.982k operations per second
Finished all jobs, totalling 6.607k operations, about 6.597k operations per second
Finished all jobs, totalling 6.83k operations, about 6.821k operations per second
Finished all jobs, totalling 5.402k operations, about 5.392k operations per second


In [6]:
# Berikut dapat dilihat contoh untuk cara threading yang kedua:
!python3 src/python/hello_world_mt_2.py

Finished with 0.742k operations, about 0.742k operations per second
Finished with 0.837k operations, about 0.837k operations per second
Finished with 0.725k operations, about 0.725k operations per second
Finished with 0.736k operations, about 0.736k operations per second
Finished with 0.856k operations, about 0.856k operations per second
Finished with 0.841k operations, about 0.841k operations per second
Finished all jobs, totalling 4.737k operations, about 4.722k operations per second


In [7]:
%%timeit -r 3 -n 3 -q
!python3 src/python/hello_world_mt_2.py -q

Finished all jobs, totalling 7.245k operations, about 7.235k operations per second
Finished all jobs, totalling 6.588k operations, about 6.580k operations per second
Finished all jobs, totalling 7.192k operations, about 7.177k operations per second
Finished all jobs, totalling 6.646k operations, about 6.638k operations per second
Finished all jobs, totalling 6.94k operations, about 6.933k operations per second
Finished all jobs, totalling 5.342k operations, about 5.330k operations per second
Finished all jobs, totalling 8.282k operations, about 8.272k operations per second
Finished all jobs, totalling 6.092k operations, about 6.077k operations per second
Finished all jobs, totalling 3.704k operations, about 3.694k operations per second



Dari kedua contoh di atas, dapat dilihat pertukaran objek pada _multithreading_ cara pertama dapat dengan menggunakan *class attribute* (*state*), sedangkan pada cara kedua dapat menggunakan `queue.Queue`.  
Seluruh objek dalam modul `queue` bersifat *thread-safe*, yaitu objek dapat dengan aman digunakan pada konteks *multithreading* dan terhindar dari *race condition*. Hal ini dicapai karena objek `queue.Queue` mengimplementasikan `threading.lock`. Dokumentasi lengkap dapat diakses di https://docs.python.org/3.6/library/queue.html#module-queue.

Versi *multithreading* lebih lambat jika dibandingkan dengan versi *sequential*. Hal ini benar dikarenakan *task* program ini CPU-bound, *multithreading* hanya menggunakan 1 *core* CPU, dan adanya *context switching* saat CPU beralih eksekusi dari satu *thread* ke *thread* yang lain.  
Seperti yang sudah disampaikan sebelumnya, *multithreading* pada CPython lebih cocok difungsikan untuk program I/O-bound.

In [8]:
### MANDATORY ###
# Before starting, run from `src/python` run `server.py` using your terminal out of this notebook
# to run Flask app that serve cats image and kitten video for testing

In [9]:
# Versi sekuensial untuk perbandingan
!python3 src/python/hello_world_io.py

Finished all jobs in 626.84 ms


In [10]:
%%timeit -r 3 -n 3 -q
!python3 src/python/hello_world_io.py

Finished all jobs in 629.11 ms
Finished all jobs in 632.06 ms
Finished all jobs in 652.94 ms
Finished all jobs in 618.27 ms
Finished all jobs in 636.24 ms
Finished all jobs in 616.03 ms
Finished all jobs in 644.61 ms
Finished all jobs in 620.32 ms
Finished all jobs in 638.38 ms


In [11]:
# Versi multithreaded
!python3 src/python/hello_world_io_mt.py

Finished all jobs in 1046.04 ms


In [12]:
%%timeit -r 3 -n 3 -q
!python3 src/python/hello_world_io_mt.py

Finished all jobs in 944.70 ms
Finished all jobs in 1122.11 ms
Finished all jobs in 984.80 ms
Finished all jobs in 969.70 ms
Finished all jobs in 922.03 ms
Finished all jobs in 928.20 ms
Finished all jobs in 929.79 ms
Finished all jobs in 940.19 ms
Finished all jobs in 967.86 ms


*Weh*, versi *multithreading* lebih lama dibandingkan dengan versi sekuensial untuk mendapatkan konten dari web? *Ono opo iki?*

<img src='src/images/Cost-Of-Context-Switching.png' width='640px'/>
<p style="text-align: center;color:gray">Gambar 4. <i>Cost of context switching</i>.<br /><i>(Sumber: <a href="https://www.codeproject.com/Articles/1083787/Tasks-and-Task-Parallel-Library-TPL-Multi-threadin">https://www.codeproject.com/Articles/1083787/Tasks-and-Task-Parallel-Library-TPL-Multi-threadin</a>)</i></p>

Hal ini benar adanya karena 3 hal:
1. Adanya *context switching*, dimana untuk melakukan *switch* dibutuhkan waktu. Dapat dilihat dari total waktu yang dibutuhkan untuk menyelesaikan seluruh pekerjaan.
2. *Switching* menggunakan *time slice* (*pre-emptive multitasking*). Dapat dilihat dari waktu masing-masing task seluruhnya menjadi lebih lama jika dibandingkan dengan versi sekuensial.
3. Proses `request` berhenti saat *thread* di mana `request` tersebut berada berhenti melakukan eksekusi (berhenti mengunduh konten web) akibat *pre-emptive multitasking*. Dapat dilihat dari waktu masing-masing task seluruhnya menjadi lebih lama jika dibandingkan dengan versi sekuensial.

Dari hal-hal tersebut diketahui startegi *multithreading* dibandingkan dengan *sequential* kurang diminati karena dibutuhkan waktu yang lebih banyak untuk menyelesaikan seluruh pekerjaan.

## Asyncio in CPython
_Package_ `asyncio` merupakan *built-in package* yang dapat digunakan untuk membuat program *single threaded concurrent*. *Package* ini memang utamanya dibuat untuk *asynchronous I/O programming*. Beberapa konsep penting yang dibawakan oleh *package* ini dan wajib dipahami sebelum membuat program *single threaded concurrent* diantaranya *event loop*, *coroutine*, *future*, dan *callback*.

*Event loop* merupakan perangkat eksekusi utama yang disediakan oleh `asyncio`, tempat di mana seluruh *task* berjalan. *Method* `asyncio.get_event_loop()` dapat digunakan untuk mendapatkan *event loop* yang telah dibuat dan diatur menjadi *event loop* yang digunakan untuk eksekusi. Jika belum ada *event loop* yang dibuat dan diatur sebelumnya, maka secara standar bawaan *package*, *mehod* tersebut akan membuat *event loop* baru dan menjadikannya sebagai *event loop* yang digunakan untuk eksekusi.

```python
import asyncio

async def async_main():
    ...

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(async_main())
    loop.close()
```

*Coroutine* merupakan *function* dalam Python yang didefinisikan menggunakan *keyword* `async def` yang dapat menghentikan eksekusi kodenya secara kooperatif sebelum mencapai `return`. *Keyword* `await` digunakan pada *coroutine function* dalam *coroutine* untuk memberikan tanda bahwa eksekusi *coroutine* tersebut dapat dihentikan hingga didapatkan hasil dari *coroutine function* yang dipanggil dan pada saat itu juga dapat mengeksekusi *coroutine* yang lain.

```python
async def f():
    ...
    return result


async def g():
    ...
    fut = f()
    r = await fut # berhenti di sini, eksekusi yang lain, dan kembali ke g() saat didapatkan hasil dari f()
    ...
    return r

async def h():
    ...
    return result
```

*Future* merupakan objek yang didapatkan saat memanggil *coroutine function*. Saat *function* `f()` dipanggil, belum terjadi apa-apa, `f()` belum dieksekusi, dan yang didapatkan adalah objek *future*. Diperlukan `await` untuk mendapatkan hasil dari *future* tersebut.

In [13]:
### MANDATORY ###
# Before starting, run from `src/python` run `server.py` using your terminal out of this notebook
# to run Flask app that serve cats image and kitten video for testing
# Ignore if it is already running

In [14]:
# Versi asyncio
!python3 src/python/hello_world_io_asyncio.py

Finished all jobs in 608.99 ms


In [15]:
%%timeit -r 3 -n 3 -q
!python3 src/python/hello_world_io_asyncio.py

Finished all jobs in 640.62 ms
Finished all jobs in 664.96 ms
Finished all jobs in 597.71 ms
Finished all jobs in 660.08 ms
Finished all jobs in 620.30 ms
Finished all jobs in 622.27 ms
Finished all jobs in 631.42 ms
Finished all jobs in 661.55 ms
Finished all jobs in 613.90 ms


Dari contoh di atas dapat dilihat pendekatan `asyncio` rata-rata lebih cepat jika dibandingkan dengan pendekatan versi sekuensial.

## Multiprocessing in CPython
*Package* `multiprocessing` merupakan *built-in package* yang dapat digunakan untuk mencapai *parallelism*. Kita dapat memanfaatkan banyak CPU *core* dengan menggunakan `multiprocessing.Process` dan `multiprocessing.Pool`, terutama untuk program CPU-bound. *Multiprocessing* bekerja dengan cara setiap masing-masing *process* diberikan *interpreter* sendiri-sendiri, sehingga antar proses dapat berjalan secara paralel. Antar proses tidak lagi terbatasi oleh GIL seperti pada pendekatan menggunakan `threading`, namun masih tetap ada GIL yang terpisah dalam setiap proses.

Perlu diketahui, terdapat perbedaan cara *default* untuk memulai *process* / *start method* antara platform Windows dan Unix. Secara garis besar, pada Windows digunakan *spawn*, sedangkan pada Unix digunakan *fork*. Dokumentasi lengkapnya dapat diakses di https://docs.python.org/3.6/library/multiprocessing.html#contexts-and-start-methods .

Masing-masing *process* berjalan menggunakan interpreter sendiri-sendiri yang berarti masing-masing interpreter memiliki ruang memori yang terpisah. Proses pertukaran obyek dan sinkronisasi *state* pada `multiprocessing` tidak semudah pada `threading` ataupun `asyncio`.

Terdapat 2 cara untuk mengkomunikasikan antar *process* yang disediakan oleh *package* `multiprocessing`:
1. `multiprocessing.Queue`  
   Mirip seperti `Queue` yang ada pada *package* `threading`. Pada dasarnya `multiprocessing.Queue` diimplementasikan menggunakan `Pipe` dan beberapa *lock/semaphore*. `multiprocessing.Queue` *thread-safe* dan *process-safe*. Dokumentasi lengkap dapat dilihat di https://docs.python.org/3.6/library/multiprocessing.html#multiprocessing.Queue .
   
<img src='src/images/multiprocessing-python-4.png' width='320px'/>
<p style="text-align: center;color:gray">Gambar 5. multiprocessing.Queue.<br /><i>(Sumber: <a href="https://www.geeksforgeeks.org/multiprocessing-python-set-2/">https://www.geeksforgeeks.org/multiprocessing-python-set-2/</a>)</i></p>

2. `multiprocessing.Pipe`  
    Pada dasarnya berbentuk koneksi yang terdiri dari 2 ujung untuk menghubungkan 2 `process`. `multiprocessing.Pipe` aman digunakan untuk `multiprocessing` selama kedua `process` mengakses ujung yang berbeda. Dokumentasi lengkap dapat dilihat di https://docs.python.org/3.6/library/multiprocessing.html#multiprocessing.Pipe .

<img src='src/images/multiprocessing-python-5.png' width='640px'/>
<p style="text-align: center;color:gray">Gambar 6. multiprocessing.Pipe.<br /><i>(Sumber: <a href="https://www.geeksforgeeks.org/multiprocessing-python-set-2/">https://www.geeksforgeeks.org/multiprocessing-python-set-2/</a>)</i></p>

Terdapat 2 cara untuk mensinkronisasikan *state* yang disediakan oleh *package* `multiprocessing`:
1. *Shared memory*  
    Data dapat disimpan dalam *shared memory* bentuk `multiprocessing.Value` atau `multiprocessing.Array`. *Shared memory* lebih cepat untuk diakses daripada *server process*, namun tipe data yang didukung hanya terbatas pada <a href="https://docs.python.org/3.6/library/ctypes.html#module-ctypes">ctypes</a>.

<img src='src/images/multiprocessing-python-2.png' width='320px'/>
<p style="text-align: center;color:gray">Gambar 7. <i>Shared memory</i>.<br /><i>(Sumber: <a href="https://www.geeksforgeeks.org/multiprocessing-python-set-2/">https://www.geeksforgeeks.org/multiprocessing-python-set-2/</a>)</i></p>

2. *Server process*  
    Bisa didapatkan dengan menggunakan `multiprocessing.Manager`, dimana dapat menampung tipe objek Python yang lebih banyak, di antaranya list, dict, Namespace, Lock, RLock, Semaphore, BoundedSemaphore, Condition, Event, Barrier, Queue, Value and Array. *Server process* lebih lambat daripada *shared memory* karena objek diakses menggunakan *proxy*. *Server process* ini juga dapat diakses oleh proses lain pada mesin yang berbeda melalui jaringan.

<img src='src/images/multiprocessing-python-3.png' width='480px'/>
<p style="text-align: center;color:gray">Gambar 8. <i>Server process</i>.<br /><i>(Sumber: <a href="https://www.geeksforgeeks.org/multiprocessing-python-set-2/">https://www.geeksforgeeks.org/multiprocessing-python-set-2/</a>)</i></p>

Pemrograman menggunakan `multiprocessing` sangat dianjurkan untuk mengikuti *programming guidelines* yang dapat diakses di https://docs.python.org/3.6/library/multiprocessing.html#programming-guidelines .

Terdapat 2 cara untuk memulai *process*:
1. Menimpa method `run()` dari sub-kelas `multiprocessing.Process` (*class inheritance*)
2. Menggunakan `multiprocessing.Pool`.

In [16]:
# Berikut dapat dilihat cara multiprocessing yang pertama
!python3 src/python/hello_world_mp.py

HelloWorldMP-1 finished with 5.633k operations, about 5.633k operations per second
HelloWorldMP-2 finished with 6.002k operations, about 6.002k operations per second
HelloWorldMP-3 finished with 5.97k operations, about 5.970k operations per second
HelloWorldMP-4 finished with 6.001k operations, about 6.001k operations per second
HelloWorldMP-5 finished with 5.979k operations, about 5.979k operations per second
HelloWorldMP-6 finished with 5.615k operations, about 5.615k operations per second
Finished all jobs, totalling 35.2k operations, about 34.892k operations per second


In [17]:
%%timeit -r 3 -n 3 -q
!python3 src/python/hello_world_mp.py -q

Finished all jobs, totalling 35.783k operations, about 35.468k operations per second
Finished all jobs, totalling 36.067k operations, about 35.757k operations per second
Finished all jobs, totalling 35.864k operations, about 35.561k operations per second
Finished all jobs, totalling 36.516k operations, about 36.206k operations per second
Finished all jobs, totalling 36.234k operations, about 35.920k operations per second
Finished all jobs, totalling 36.406k operations, about 36.097k operations per second
Finished all jobs, totalling 36.125k operations, about 35.775k operations per second
Finished all jobs, totalling 35.633k operations, about 35.299k operations per second
Finished all jobs, totalling 35.858k operations, about 35.531k operations per second


In [18]:
# Berikut dapat dilihat cara multiprocessing yang kedua
!python3 src/python/hello_world_mp_pool.py

Worker 0: Finished with 6.18k operations, about 6.180k operations per second
Worker 2: Finished with 6.093k operations, about 6.093k operations per second
Worker 1: Finished with 5.962k operations, about 5.962k operations per second
Worker 3: Finished with 6.157k operations, about 6.157k operations per second
Worker 4: Finished with 6.199k operations, about 6.199k operations per second
Worker 5: Finished with 6.02k operations, about 6.020k operations per second
Finished all jobs, totalling 36.611k operations, about 36.265k operations per second


In [19]:
%%timeit -r 3 -n 3 -q
!python3 src/python/hello_world_mp_pool.py -q

Finished all jobs, totalling 36.491k operations, about 36.144k operations per second
Finished all jobs, totalling 37.072k operations, about 36.720k operations per second
Finished all jobs, totalling 36.797k operations, about 36.450k operations per second
Finished all jobs, totalling 35.896k operations, about 35.567k operations per second
Finished all jobs, totalling 36.475k operations, about 36.129k operations per second
Finished all jobs, totalling 36.413k operations, about 36.048k operations per second
Finished all jobs, totalling 36.496k operations, about 36.148k operations per second
Finished all jobs, totalling 36.863k operations, about 36.512k operations per second
Finished all jobs, totalling 36.379k operations, about 36.034k operations per second


Dari kedua contoh di atas, diketahui versi *multiprocessing* jauh lebih cepat jika dibandingkan dengan versi sekuensial. Hal ini benar dikarenakan *task* program ini CPU-bound, *multiprocessing* menggunakan *core* CPU sejumlah `num_jobs` / *worker* yang diminta. Jumlah *worker* ini bervariasi, tergantung jumlah *core* dari CPU. *Rule-of-thumb* yang dapat digunakan adalah jumlah *worker* sama dengan jumlah *physical cores*.  

In [20]:
# Multiprocessing untuk I/O
!python3 src/python/hello_world_io_mp_pool.py

Finished all jobs in 1043.79 ms


In [21]:
%%timeit -r 3 -n 3 -q
!python3 src/python/hello_world_io_mp_pool.py

Finished all jobs in 1078.99 ms
Finished all jobs in 1025.04 ms
Finished all jobs in 974.27 ms
Finished all jobs in 1004.44 ms
Finished all jobs in 1084.21 ms
Finished all jobs in 1183.79 ms
Finished all jobs in 1207.08 ms
Finished all jobs in 1203.11 ms
Finished all jobs in 1226.56 ms


Hasil *multiprocessing* pada contoh program I/O-bound di atas, waktu eksekusi lebih lambat dibandingkan dengan versi sekuensial dikarenakan adanya *overhead* saat membuat *process* baru. Selain itu, modul `requests` sebenarnya tidak dapat dibagikan dan digunakan secara bersama-sama, namun masing-masing proses membuat salinan objek baru sendiri-sendiri di dalam proses.

## Summary

<img src='src/images/summary.png' width='1280px'/>
<p style="text-align: center;color:gray">Gambar 9. <i>Choosing concurrency and parallelism for your Python projects.</i></p>

### Concurrency and parallelism packages
Banyak sekali `packages` yang dikembangkan oleh pihak ketiga dengan menggunakan berbagai macam teknologi agar dicapai *concurrency* dan *parallelism*. Beberapa diantaranya yang dapat saya sebutkan:  
Twisted, Dask, Celery, Ray, IOLoop, Gevent, dan masih banyak lagi.

### References
https://hackernoon.com/why-concurrency-is-hard-a45295e96114  
https://stackoverflow.com/questions/1050222/what-is-the-difference-between-concurrency-and-parallelism  
https://realpython.com/python-concurrency/  
https://docs.python-guide.org/starting/which-python/  
https://www.jython.org/jythonbook/en/1.0/Concurrency.html  
https://hackernoon.com/concurrent-programming-in-python-is-not-what-you-think-it-is-b6439c3f3e6a  
https://whatis.techtarget.com/definition/preemptive-multitasking  
https://www.pcmag.com/encyclopedia/term/48051/non-preemptive-multitasking  
https://stackoverflow.com/questions/3042717/what-is-the-difference-between-a-thread-process-task  
https://www.hellsoft.se/understanding-cpu-and-i-o-bound-for-asynchronous-operations/  
https://realpython.com/python-gil/  
https://docs.python.org/3.6/library/threading.html  
https://docs.python.org/3.6/library/multiprocessing.html  
https://docs.python.org/3.6/library/asyncio.html  
https://docs.python.org/3.6/library/queue.html  